In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
from itertools import combinations, groupby
from collections import Counter
from pathlib import Path
from apriori_algo import AprioriAlgo
apriori = AprioriAlgo()

In [3]:
orders = pd.read_csv('../input/all_orders.csv', index_col = 0)
cluster_df = pd.read_csv('../input/cluster_df')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

c:\users\akshaysharma\hvacproakshay\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
orders = orders.sort_values(['order_id','product_id'])
orders

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,1
30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,1
30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,1
30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,1
30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,1
...,...,...,...,...,...,...,...,...,...,...
23108244,3421083,21162,3,0,92,18,baby food formula,babies,25247,1
23108238,3421083,24852,9,1,24,4,fresh fruits,produce,25247,1
23108247,3421083,35211,5,0,92,18,baby food formula,babies,25247,1
23108242,3421083,39678,6,1,74,17,dish detergents,household,25247,1


In [13]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [19]:
cluster_orders[0]

order_id
13           196
13          3800
13         17330
13         23020
13         25783
           ...  
3421078    37646
3421078    38656
3421078    40396
3421078    41844
3421078    48577
Name: item_id, Length: 1740670, dtype: int64

In [15]:
# run each cluster series through association rule function and obtain association rules at item level
REBUILD_TEST_DATA = False
test_data_path = "../input/associationRule/group0_item_rules.csv"
if REBUILD_TEST_DATA or not Path(test_data_path).exists():
    group_association_rules_dic = {}
    for x in range(0,6):
        group_association_rules_dic[x] = apriori.association_rules(cluster_orders[x],.0001)
    for x in group_association_rules_dic:
        group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
                   .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
        group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
                   .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
        group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                        'product_name_B','freqAB','supportAB','freqA',
                                                                        'supportA','freqB','supportB','confidenceAtoB',
                                                                        'confidenceBtoA','lift']]
    for x in group_association_rules_dic:
        print("Writing cluster " + str(x) + "'s item rules to csv...")
        group_association_rules_dic[x].to_csv('../input/associationRule/group'+str(x)+'_item_rules.csv')


Starting order_item:                1740670
Items with support >= 0.0001:            6216
Remaining order_item:               1631820
Remaining orders with 2+ items:      225372
Remaining order_item:               1609574
Item pairs:                         1643666
Item pairs with support >= 0.0001:      45408

Starting order_item:               15524044
Items with support >= 0.0001:           10316
Remaining order_item:              14600167
Remaining orders with 2+ items:     1276870
Remaining order_item:              14560949
Item pairs:                        14030084
Item pairs with support >= 0.0001:      95546

Starting order_item:               10153348
Items with support >= 0.0001:           13288
Remaining order_item:               9206509
Remaining orders with 2+ items:      992409
Remaining order_item:               9130882
Item pairs:                        15999908
Item pairs with support >= 0.0001:      28714

Starting order_item:                5642242
Items with suppor

In [20]:
group_association_rules_dic[0]

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,4985,36278,Homestyle Some Pulp Orange Juice,Chocolate Breakfast Biscuits Chocolate,27,0.000120,30,0.000133,32,0.000142,0.900000,0.843750,6338.587500
1,4985,48927,Homestyle Some Pulp Orange Juice,Fat Free Plain Yoghurt,24,0.000106,30,0.000133,62,0.000275,0.800000,0.387097,2908.025806
2,36278,48927,Chocolate Breakfast Biscuits Chocolate,Fat Free Plain Yoghurt,25,0.000111,32,0.000142,62,0.000275,0.781250,0.403226,2839.868952
3,24852,48927,Banana,Fat Free Plain Yoghurt,32,0.000142,51126,0.226852,62,0.000275,0.000626,0.516129,2.275184
4,38259,38576,Blueberry Greek Yogurt + Chia,The Epic Seed Greek Yogurt Strawberry & Chia,23,0.000102,33,0.000146,25,0.000111,0.696970,0.920000,6283.098182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45403,21137,45374,Organic Strawberries,Newman O's Creme Filled Chocolate Cookies,30,0.000133,31351,0.139108,106,0.000470,0.000957,0.283019,2.034529
45404,21137,34282,Organic Strawberries,Original Sandwich Bread,23,0.000102,31351,0.139108,84,0.000373,0.000734,0.273810,1.968326
45405,21137,37774,Organic Strawberries,Artesian Sparkling Water,24,0.000106,31351,0.139108,90,0.000399,0.000766,0.266667,1.916979
45406,21137,49174,Organic Strawberries,Raincoast Crisps Cranberry And Hazelnut Crackers,27,0.000120,31351,0.139108,106,0.000470,0.000861,0.254717,1.831076


In [7]:
i1 = orders.set_index('order_id')['product_id'].rename('item_id')

In [8]:
i1.head()

order_id
1    10246
1    11109
1    13176
1    22035
1    43633
Name: item_id, dtype: int64

In [9]:
association_rule_no_clustering = apriori.association_rules(i1,.0001)

Starting order_item:               33819106
Items with support >= 0.0001:           10949
Remaining order_item:              31116451
Remaining orders with 2+ items:     3136021
Remaining order_item:              30928016
Item pairs:                        21061284
Item pairs with support >= 0.0001:      61832



In [11]:
association_rule_no_clustering

,item_A,item_B,aisle_name_A,aisle_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift


In [10]:
association_rule_no_clustering = association_rule_no_clustering.merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
association_rule_no_clustering = association_rule_no_clustering.merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
association_rule_no_clustering = association_rule_no_clustering[['item_A','item_B','aisle_name_A',
                                                                        'aisle_name_B','freqAB','supportAB','freqA',
                                                                        'supportA','freqB','supportB','confidenceAtoB',
                                                                        'confidenceBtoA','lift']]
print("Writing clusters aisle rules to csv...")
association_rule_no_clustering.to_csv('../input/associationRule_no_cluster/all_aisle_rules.csv')

Writing clusters aisle rules to csv...


In [17]:
# separate order data by cluster and transform into expected format of group association function at aisle level

cluster_aisle_orders = {}
for i in range(0,6):
    x = orders[orders['cluster'] == i]
    x = x.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[i] = x

In [18]:
# format item association rule dataframes and merge product names
REBUILD_TEST_DATA = False
test_data_path = "../input/associationRule/group0_aisle_rules.csv"
if REBUILD_TEST_DATA or not Path(test_data_path).exists():
    group_aisle_association_rules_dic = {}
    for x in range(0,6):
        group_aisle_association_rules_dic[x] = apriori.association_rules(cluster_aisle_orders[x],.0001)
    for x in group_aisle_association_rules_dic:
        group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
        group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
                   .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
        group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                        'aisle_name_B','freqAB','supportAB','freqA',
                                                                        'supportA','freqB','supportB','confidenceAtoB',
                                                                        'confidenceBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})
    for x in group_aisle_association_rules_dic:
        print("Writing cluster " + str(x) + "'s aisle rules to csv...")
        group_aisle_association_rules_dic[x].to_csv('../input/associationRule/group'+str(x)+'_aisle_rules.csv')

Starting order_item:                1184688
Items with support >= 0.0001:             134
Remaining order_item:               1184688
Remaining orders with 2+ items:      219857
Remaining order_item:               1155194
Item pairs:                            8554
Item pairs with support >= 0.0001:       5396

Starting order_item:               11211683
Items with support >= 0.0001:             134
Remaining order_item:              11211683
Remaining orders with 2+ items:     1281160
Remaining order_item:              11171978
Item pairs:                            8906
Item pairs with support >= 0.0001:       7573

Starting order_item:                7680091
Items with support >= 0.0001:             134
Remaining order_item:               7680091
Remaining orders with 2+ items:      991382
Remaining order_item:               7595005
Item pairs:                            8908
Item pairs with support >= 0.0001:       8202

Starting order_item:                3694150
Items with suppor

True